### Process Fitness Data

##### Preprocess Data:
* Extract recorded activities
* Create uids
* Create max RMs

##### Pull Data

In [1]:
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
drive = GoogleDrive(gauth)
drive?

Type:           GoogleDrive
String form:    <pydrive.drive.GoogleDrive object at 0x0000023864986A08>
File:           c:\python37\lib\site-packages\pydrive\drive.py
Docstring:      Main Google Drive class.
Init docstring:
Create an instance of GoogleDrive.

:param auth: authorized GoogleAuth instance.
:type auth: pydrive.auth.GoogleAuth.


In [2]:
mimetypes = {
    # Drive Document files as MS Word files.
    'application/vnd.google-apps.document': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',

    # Drive Sheets files as MS Excel files.
    'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'

    # etc.
}

file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

for file1 in file_list:
    
    if file1['title'] == "FY20 H1 Workout Tracker" :
        
        download_mimetype = None
        if file1['mimeType'] in mimetypes:
            download_mimetype = mimetypes[file1['mimeType']]

        
        file1.GetContentFile("tmp_"+file1['title']+".xlsx", mimetype=download_mimetype)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=874659251868-8fdkk74gtuje4j65gvjosq2gcu447hga.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
import pandas as pd
import datetime
import pprint

file = "tmp_FY20 H1 Workout Tracker.xlsx"

df = pd.read_excel(file, skiprows=3)
print(df.shape)
df.head(3)

(101, 13)


,Rotation,1RM,Workout,Exercise,Week,Sets,Reps,% 1RM (pace if running),Actual Lift,Date,Time,Bodyweight,Notes
0,3.0,65.17,1.0,Shoulder Press,1.0,3.0,5,"65, 75, 85",4x55,2019-10-24,14:00:00,88.7,NaN
1,NaN,NaN,NaN,Shoulder Press,2.0,3.0,3,"70, 80, 90",NaN,NaT,NaN,NaN,NaN
2,NaN,NaN,NaN,Shoulder Press,3.0,3.0,5 / 3 / 1,"75, 85, 95",NaN,NaT,NaN,NaN,NaN


#### Preprocessing

In [4]:
# split out powerlifting, accessory, and running rows
recorded_exercise = [
    'Shoulder Press',
    'Bench Press',
    'Deadlifts',
    'Squat',
    'Run'
]
df['map'] = df['Exercise'].apply(
    lambda x: True if x in recorded_exercise and pd.notnull(x) else False
)
rec_df = df[df['map']==True].drop('map', axis=1)
print("Total exercises:",len(rec_df))


# drop unfilled nan rows
rec_df = rec_df.dropna(
    how='any',
    subset=['Date']
)
print("Total recorded exercises:",len(rec_df))
print(rec_df['Exercise'].value_counts())

Total exercises: 63
Total recorded exercises: 46
Run               11
Shoulder Press     9
Bench Press        9
Deadlifts          9
Squat              8
Name: Exercise, dtype: int64


In [5]:
# propogate Rotation, and Workout down, create new sub_Workout column -> have a 3 digit code to identify any workout eg 2_1.2 
rec_df[['Rotation', 'Workout']] = rec_df[['Rotation', 'Workout']].fillna(method='ffill')

def make_uid(row) : 
    return str(int(row['Rotation'])) + "_" + str(row['Workout']) + "." + str(int(row['Week']))
rec_df['uid'] = rec_df.apply(
    make_uid, axis=1
)

rec_df.set_index('uid', inplace=True)

# calculate theoritcal_1RM from Actual lift
def make_1rm(r) :
    
    if r['Exercise'] != 'Run':
        x = r['Actual Lift']
        r, w = x.split('x')[0], x.split('x')[1]
        return float(w)*(1 + (int(r)/30))
    
    return None # find a better proxy

rec_df['calculated_1RM'] = rec_df.apply(
    make_1rm, axis=1
)

# process runnning data
rec_df['Distance'], rec_df['Pace'], rec_df['% 1RM'] = None, None, None
def make_running(r) :
    
    if r['Exercise'] == 'Run' :
        pace_numeric = r['% 1RM (pace if running)'].hour + r['% 1RM (pace if running)'].minute * 0.01
        return float(r['Actual Lift'].split("km")[0]), r['% 1RM (pace if running)'], pace_numeric, None, None
    else :
        return None, None, None, r['Actual Lift'], r['% 1RM (pace if running)']
    
rec_df[['Distance', 'Pace', 'Pace Numeric', 'Actual Lift', '% 1RM']] = rec_df.apply(make_running, axis=1, result_type="expand")
rec_df = rec_df.drop('% 1RM (pace if running)', axis=1)

rec_df.head(2)

,Rotation,1RM,Workout,Exercise,Week,Sets,Reps,Actual Lift,Date,Time,Bodyweight,Notes,calculated_1RM,Distance,Pace,% 1RM,Pace Numeric
uid,,,,,,,,,,,,,,,,,
3_1.0.1,3.0,65.17,1.0,Shoulder Press,1.0,3.0,5,4x55,2019-10-24,14:00:00,88.7,NaN,62.333333,NaN,None,"65, 75, 85",NaN
3_2.0.1,3.0,140,2.0,Deadlifts,1.0,3.0,5,5x120,2019-10-26,16:00:00,88.8,NaN,140.000000,NaN,None,"65, 75, 85",NaN


#### Write out

In [6]:
rec_df.to_excel("Workout_Tracker_Processed.xlsx")

In [7]:
rec_df['Exercise'].value_counts()

Run               11
Shoulder Press     9
Bench Press        9
Deadlifts          9
Squat              8
Name: Exercise, dtype: int64